In [0]:
CREATE CATALOG IF NOT EXISTS retail_catalog;
CREATE SCHEMA IF NOT EXISTS retail_catalog.bronze_schema;

In [0]:
CREATE TABLE IF NOT EXISTS retail_catalog.bronze_schema.customers_bronze
USING DELTA
LOCATION 'abfss://retailstream360@storageaccountsudip1.dfs.core.windows.net/bronze/customers';

CREATE TABLE IF NOT EXISTS retail_catalog.bronze_schema.products_bronze
USING DELTA
LOCATION 'abfss://bronze@<account>.dfs.core.windows.net/retail/bronze/products/';

CREATE TABLE IF NOT EXISTS retail_catalog.bronze_schema.orders_bronze
USING DELTA
LOCATION 'abfss://bronze@<account>.dfs.core.windows.net/retail/bronze/orders/';

CREATE TABLE IF NOT EXISTS retail_catalog.bronze_schema.transactions_bronze
USING DELTA
LOCATION 'abfss://bronze@<account>.dfs.core.windows.net/retail/bronze/transactions/';


In [0]:
%python
#checkpoint_path = f"{checkpoint_base_path}/{table_name}"
checkpoint_path = "abfss://retailstream360@storageaccountsudip1.dfs.core.windows.net/bronze/checkpoints/customers/"
file_format = "csv"
raw_path = "abfss://retailstream360@storageaccountsudip1.dfs.core.windows.net/raw/customers/"

from pyspark.sql.functions import current_timestamp, input_file_name

#bronze_path = f"{bronze_base_path}/{table_name}"
bronze_path = "abfss://retailstream360@storageaccountsudip1.dfs.core.windows.net/bronze/customers"


In [0]:
%python

df = (
        spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", file_format)
        .option("cloudFiles.schemaLocation", checkpoint_path + "/schema")
        .option("cloudFiles.inferColumnTypes", "true")
        .load(raw_path)
        .withColumn("_ingest_time", current_timestamp())
        .withColumn("_source_file", input_file_name())
        .writeStream
        .format("delta")
        .option("checkpointLocation", checkpoint_path)
        .outputMode("append")
        .trigger(availableNow=True)
        .start(bronze_path)
    )
#df.stop()